In [14]:
# !pip install Box2D
# !pip install box2d-py
# !pip install gym[all]
# !pip install gym[Box_2D]
# !pip install torc
# !pip install -U "ray[rllib]" torch
import pickle as pkl
import gym 
env = gym.make("LunarLander-v2")

In [6]:
import ray
import ray.rllib.agents.ppo as ppo
from ray.tune.logger import pretty_print

config = ppo.DEFAULT_CONFIG.copy()
config["num_gpus"] = 0
config["num_workers"] = 1
trainer = ppo.PPOTrainer(config=config, env="LunarLander-v2")

# Can optionally call trainer.restore(path) to load a checkpoint.

for i in range(2):
   # Perform one iteration of training the policy with PPO
   result = trainer.train()
   print(pretty_print(result))

   if i % 2 == 0:
       checkpoint = trainer.save()
       print("checkpoint saved at", checkpoint)

# Also, in case you have trained a model outside of ray/RLlib and have created
# an h5-file with weight values in it, e.g.
# my_keras_model_trained_outside_rllib.save_weights("model.h5")
# (see: https://keras.io/models/about-keras-models/)

# ... you can load the h5-weights into your Trainer's Policy's ModelV2
# (tf or torch) by doing:

# NOTE: In order for this to work, your (custom) model needs to implement
# the `import_from_h5` method.
# See https://github.com/ray-project/ray/blob/master/rllib/tests/test_model_imports.py
# for detailed examples for tf- and torch trainers/models.

2022-04-18 16:49:03,979	INFO trainable.py:127 -- Trainable.setup took 13.890 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2022-04-18 16:49:03,982	WARNING util.py:55 -- Install gputil for GPU system monitoring.


agent_timesteps_total: 4000
custom_metrics: {}
date: 2022-04-18_16-49-12
done: false
episode_len_mean: 94.38095238095238
episode_media: {}
episode_reward_max: -49.521158445305524
episode_reward_mean: -176.6025630053101
episode_reward_min: -460.596531980081
episodes_this_iter: 42
episodes_total: 42
experiment_id: 1388888b1443412ebd4aa5949d8357c4
hostname: STEFANXPS
info:
  learner:
    default_policy:
      custom_metrics: {}
      learner_stats:
        cur_kl_coeff: 0.20000000298023224
        cur_lr: 4.999999873689376e-05
        entropy: 1.3703820705413818
        entropy_coeff: 0.0
        kl: 0.016125040128827095
        model: {}
        policy_loss: -0.006196949630975723
        total_loss: 8762.4814453125
        vf_explained_var: -0.005872588604688644
        vf_loss: 8762.484375
  num_agent_steps_sampled: 4000
  num_agent_steps_trained: 4000
  num_steps_sampled: 4000
  num_steps_trained: 4000
  num_steps_trained_this_iter: 4000
iterations_since_restore: 1
node_ip: 127.0.0.1
n

In [11]:
import ray
from ray import tune

analysis = tune.run(
    "PPO",
    metric="episode_reward_mean",
    mode="max",
    stop={"training_iteration": 3},
    config={
        "env": "LunarLander-v2",
        "num_gpus": 0,
        #"num_workers": 1,
        "lr": tune.grid_search([0.01, 0.001, 0.0001]),
    },
    verbose=1,
)

2022-04-18 16:55:41,287	INFO tune.py:639 -- Total run time: 86.44 seconds (86.18 seconds for the tuning loop).


In [12]:
print("Best hyperparameters found were: ", analysis.best_config)

Best hyperparameters found were:  {'env': 'LunarLander-v2', 'num_gpus': 0, 'lr': 0.001}


In [27]:
analysis.dataframe(metric="episode_reward_mean", mode="max")[['episode_reward_mean', 'episode_reward_max', 'config/lr']]

,episode_reward_mean,episode_reward_max,config/lr
0,-183.608621,-6.156078,0.0100
1,-123.204116,31.522609,0.0010
2,-124.371005,-8.912454,0.0001


In [29]:
# save analysis to file
with open("analysis_finalDFs_PPO.pkl", "wb") as f:
    pkl.dump(analysis.dataframe(), f)

with open("analysis_trialDFs_PPO.pkl", "wb") as f:
    pkl.dump(analysis.trial_dataframes, f)

with open("analysis_configs_PPO.pkl", "wb") as f:
    pkl.dump(analysis._configs, f)